# **Real-ESRGAN Upscaler - 2x EVALUATION MODE (Fast)**

## Purpose
This notebook upscales images to **2x the target print size** for quality evaluation in Photoshop.

- 11x14" target → outputs at 22x28" for inspection
- 10x20" target → outputs at 20x40" for inspection

## Speed Comparison
- **This notebook**: ~30-120 seconds per image (fast evaluation)
- **FLUX notebook**: ~20-40 minutes per image (higher quality)

## When to Use
- Quick quality checks before final FLUX upscale
- Preview pixelation at full print size
- Fast iteration on multiple images

## Setup Instructions
1. Run the **Setup Environment** cell first (only needed once)
2. Set your `IMAGE_CONFIGS` with the **original target sizes** (not 2x)
3. The notebook automatically calculates 2x dimensions
4. Run the **Upscale** cell

In [1]:
# @title Setup Environment (Run Once)
import subprocess
import sys
import os
from pathlib import Path

# Get repo root
NOTEBOOK_DIR = Path(os.getcwd()).resolve()
if NOTEBOOK_DIR.name == "resolution-upscaling":
    REPO_ROOT = NOTEBOOK_DIR.parent
else:
    REPO_ROOT = NOTEBOOK_DIR

# Directory paths
HF_DIR = REPO_ROOT / "hf"
MODELS_DIR = HF_DIR / "models"
UPSCALE_MODELS_DIR = MODELS_DIR / "upscale_models"
OUTPUT_DIR = REPO_ROOT / "4xoutput"
INPUT_DIR = REPO_ROOT / "input"

# Create directories
for d in [HF_DIR, MODELS_DIR, UPSCALE_MODELS_DIR, OUTPUT_DIR, INPUT_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print(f"Repo root: {REPO_ROOT}")
print(f"Models directory: {UPSCALE_MODELS_DIR}")
print(f"Output directory: {OUTPUT_DIR}")

def install_packages():
    """Install required packages for Real-ESRGAN standalone."""
    # PyTorch with CUDA 12.8 for RTX 5060 Ti
    print("Installing PyTorch nightly with CUDA 12.8...")
    result = subprocess.run(
        [sys.executable, '-m', 'pip', 'install', '-q', '--pre',
         'torch', 'torchvision',
         '--index-url', 'https://download.pytorch.org/whl/nightly/cu128'],
        capture_output=True
    )
    if result.returncode == 0:
        print("✓ PyTorch installed")
    else:
        print(f"✗ PyTorch install failed: {result.stderr.decode()}")
    
    # Spandrel for loading upscale models
    packages = ['spandrel', 'opencv-python', 'huggingface_hub', 'safetensors', 'pillow']
    for package in packages:
        try:
            subprocess.run(
                [sys.executable, '-m', 'pip', 'install', '-q', package],
                check=True, capture_output=True
            )
            print(f"✓ {package} installed")
        except subprocess.CalledProcessError as e:
            print(f"✗ Error installing {package}")

print("Installing packages...")
install_packages()
print("\n✅ Environment Setup Complete!")

Repo root: C:\Users\Armaan\Desktop\Artinafti
Models directory: C:\Users\Armaan\Desktop\Artinafti\hf\models\upscale_models
Output directory: C:\Users\Armaan\Desktop\Artinafti\4xoutput
Installing packages...
Installing PyTorch nightly with CUDA 12.8...
✓ PyTorch installed
✓ spandrel installed
✓ opencv-python installed
✓ huggingface_hub installed
✓ safetensors installed
✓ pillow installed

✅ Environment Setup Complete!


In [2]:
# @title Download Models (Run Once)
from huggingface_hub import hf_hub_download
from pathlib import Path
import os

# Ensure paths are set
NOTEBOOK_DIR = Path(os.getcwd()).resolve()
if NOTEBOOK_DIR.name == "resolution-upscaling":
    REPO_ROOT = NOTEBOOK_DIR.parent
else:
    REPO_ROOT = NOTEBOOK_DIR
UPSCALE_MODELS_DIR = REPO_ROOT / "hf" / "models" / "upscale_models"
UPSCALE_MODELS_DIR.mkdir(parents=True, exist_ok=True)

def download_model(repo_id: str, filename: str, dest_dir: Path) -> str:
    """Download model from HuggingFace Hub."""
    dest_path = dest_dir / filename
    if dest_path.exists():
        print(f"✓ {filename} already exists")
        return filename
    
    try:
        print(f"Downloading {filename}...", end=' ', flush=True)
        hf_hub_download(
            repo_id=repo_id,
            filename=filename,
            local_dir=str(dest_dir),
            local_dir_use_symlinks=False
        )
        print("Done!")
        return filename
    except Exception as e:
        print(f"\nError downloading {filename}: {e}")
        return None

print("Downloading upscale models...")
download_model("Isi99999/Upscalers", "4x-UltraSharp.pth", UPSCALE_MODELS_DIR)
download_model("Isi99999/Upscalers", "4x_foolhardy_Remacri.pth", UPSCALE_MODELS_DIR)
download_model("Isi99999/Upscalers", "4x-AnimeSharp.pth", UPSCALE_MODELS_DIR)

print("\n✅ Models downloaded!")

✓ 4x-UltraSharp.pth already exists
✓ 4x_foolhardy_Remacri.pth already exists
✓ 4x-AnimeSharp.pth already exists

✅ Models downloaded!


In [3]:
# @title Load Libraries
import torch
import numpy as np
from PIL import Image
import cv2
import gc
import os
import time
from pathlib import Path
from IPython.display import display, Image as IPImage

# Import spandrel for model loading
from spandrel import ImageModelDescriptor, ModelLoader

# Set up paths
NOTEBOOK_DIR = Path(os.getcwd()).resolve()
if NOTEBOOK_DIR.name == "resolution-upscaling":
    REPO_ROOT = NOTEBOOK_DIR.parent
else:
    REPO_ROOT = NOTEBOOK_DIR

UPSCALE_MODELS_DIR = REPO_ROOT / "hf" / "models" / "upscale_models"
OUTPUT_DIR = REPO_ROOT / "4xoutput"
INPUT_DIR = REPO_ROOT / "input"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Device setup
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def clear_memory():
    """Clear GPU memory."""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
    gc.collect()

print(f"✅ Libraries loaded!")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

✅ Libraries loaded!
CUDA available: True
GPU: NVIDIA GeForce RTX 5060 Ti
VRAM: 15.9 GB


In [ ]:
# @title Configuration - 2x EVALUATION MODE

# ============== 2x EVALUATION MODE ==============
# This notebook upscales to 2x the target print size for quality inspection.
# Enter your ORIGINAL target sizes below - the notebook automatically doubles them.
#
# Examples:
#   11x14" target → evaluates at 22x28"
#   10x20" target → evaluates at 20x40"

EVALUATION_MULTIPLIER = 2  # 2x target dimensions for quality testing

DPI = 150

IMAGE_CONFIGS = [
    {
        "image_id": "7a49a140-beee-4141-9593-3fddb9194240",
        "input_path": "input/7a49a140-beee-4141-9593-3fddb9194240.jpg",
        "width_inches": 20,   # ORIGINAL target (will evaluate at 40")
        "height_inches": 10,  # ORIGINAL target (will evaluate at 20")
    },
    {
        "image_id": "df5cc4ff-9846-412c-ad72-e53c42184b9c",
        "input_path": "input/df5cc4ff-9846-412c-ad72-e53c42184b9c.jpg",
        "width_inches": 20,   # ORIGINAL target (will evaluate at 40")
        "height_inches": 10,  # ORIGINAL target (will evaluate at 20")
    },
    {
        "image_id": "b46bc519-9b4b-487f-b8d4-b95195d7e02e",
        "input_path": "input/b46bc519-9b4b-487f-b8d4-b95195d7e02e.jpg",
        "width_inches": 20,   # ORIGINAL target (will evaluate at 40")
        "height_inches": 10,  # ORIGINAL target (will evaluate at 20")
    },
]

# Use 16x upscaling (two passes of 4x model)
USE_TWO_PASS = True

# Calculate 2x evaluation dimensions
for config in IMAGE_CONFIGS:
    config["original_width"] = config["width_inches"]
    config["original_height"] = config["height_inches"]
    config["eval_width_inches"] = config["width_inches"] * EVALUATION_MULTIPLIER
    config["eval_height_inches"] = config["height_inches"] * EVALUATION_MULTIPLIER
    config["target_width_px"] = config["eval_width_inches"] * DPI
    config["target_height_px"] = config["eval_height_inches"] * DPI
    config["target_aspect"] = config["width_inches"] / config["height_inches"]


def calculate_scale_for_crop(
    input_width: int,
    input_height: int,
    target_width_inches: float,
    target_height_inches: float,
    dpi: int
) -> dict:
    """Calculate aspect-ratio-aware output dimensions."""
    final_width_px = int(target_width_inches * dpi)
    final_height_px = int(target_height_inches * dpi)
    
    input_aspect = input_width / input_height
    target_aspect = target_width_inches / target_height_inches
    
    if abs(input_aspect - target_aspect) < 0.01:
        scale_factor = final_width_px / input_width
        return {
            "output_width_px": final_width_px,
            "output_height_px": final_height_px,
            "final_width_px": final_width_px,
            "final_height_px": final_height_px,
            "scale_factor": scale_factor,
            "crop_direction": "none",
            "crop_amount_px": 0,
            "crop_amount_inches": 0.0,
            "equivalent_print_size": f"{target_width_inches}\" x {target_height_inches}\""
        }
    
    if input_aspect < target_aspect:
        output_width_px = final_width_px
        scale_factor = output_width_px / input_width
        output_height_px = int(input_height * scale_factor)
        
        crop_amount_px = output_height_px - final_height_px
        crop_amount_inches = crop_amount_px / dpi
        crop_direction = "vertical"
        equivalent_height_inches = output_height_px / dpi
        equivalent_print_size = f"{target_width_inches}\" x {equivalent_height_inches:.2f}\""
    else:
        output_height_px = final_height_px
        scale_factor = output_height_px / input_height
        output_width_px = int(input_width * scale_factor)
        
        crop_amount_px = output_width_px - final_width_px
        crop_amount_inches = crop_amount_px / dpi
        crop_direction = "horizontal"
        equivalent_width_inches = output_width_px / dpi
        equivalent_print_size = f"{equivalent_width_inches:.2f}\" x {target_height_inches}\""
    
    return {
        "output_width_px": output_width_px,
        "output_height_px": output_height_px,
        "final_width_px": final_width_px,
        "final_height_px": final_height_px,
        "scale_factor": scale_factor,
        "crop_direction": crop_direction,
        "crop_amount_px": crop_amount_px,
        "crop_amount_inches": crop_amount_inches,
        "equivalent_print_size": equivalent_print_size
    }


# ============== OUTPUT FORMAT SETTINGS ==============
OUTPUT_FORMATS = ["png", "tiff"]

# ============== UPSCALE SETTINGS ==============
upscale_model = "4x-UltraSharp.pth"
tile_size = 512
tile_overlap = 32
use_fp16 = True

print("=" * 60)
print("2x EVALUATION MODE - Configuration")
print("=" * 60)
print(f"Model: {upscale_model}")
print(f"Two-pass (16x): {USE_TWO_PASS}")
print(f"DPI: {DPI}")
print(f"Evaluation multiplier: {EVALUATION_MULTIPLIER}x")
print(f"Output formats: {OUTPUT_FORMATS}")
print(f"\nImages to process (2x evaluation sizes):")
for config in IMAGE_CONFIGS:
    print(f"  {config['image_id'][:8]}...")
    print(f"    Original target: {config['original_width']}\" x {config['original_height']}\"")
    print(f"    Evaluation size: {config['eval_width_inches']}\" x {config['eval_height_inches']}\" @ {DPI}DPI")
    print(f"    Output pixels: {config['target_width_px']}x{config['target_height_px']}px")

2x EVALUATION MODE - Configuration
Model: 4x-UltraSharp.pth
Two-pass (16x): True
DPI: 150
Evaluation multiplier: 2x
Output formats: ['png', 'tiff']

Images to process (2x evaluation sizes):
  7a49a140...
    Original target: 20" x 10"
    Evaluation size: 40" x 20" @ 150DPI
    Output pixels: 6000x3000px
  df5cc4ff...
    Original target: 20" x 10"
    Evaluation size: 40" x 20" @ 150DPI
    Output pixels: 6000x3000px
  b46bc519...
    Original target: 20" x 10"
    Evaluation size: 40" x 20" @ 150DPI
    Output pixels: 6000x3000px


In [5]:
# @title Upscale Functions

def upscale_with_tiles(model, img_tensor: torch.Tensor, tile_size: int, tile_overlap: int) -> torch.Tensor:
    """
    Upscale image using tiled processing to handle large images.
    
    Args:
        model: Loaded spandrel model
        img_tensor: Input image tensor (B, C, H, W)
        tile_size: Size of each tile
        tile_overlap: Overlap between tiles
    
    Returns:
        Upscaled image tensor
    """
    scale = model.scale
    _, _, h, w = img_tensor.shape
    
    # If image is small enough, process directly
    if h <= tile_size and w <= tile_size:
        with torch.no_grad():
            return model(img_tensor)
    
    # Calculate output size
    out_h, out_w = h * scale, w * scale
    output = torch.zeros((1, 3, out_h, out_w), device=img_tensor.device, dtype=img_tensor.dtype)
    weight = torch.zeros((1, 1, out_h, out_w), device=img_tensor.device, dtype=img_tensor.dtype)
    
    # Calculate tile positions
    stride = tile_size - tile_overlap
    h_tiles = max(1, (h - tile_overlap) // stride + (1 if (h - tile_overlap) % stride else 0))
    w_tiles = max(1, (w - tile_overlap) // stride + (1 if (w - tile_overlap) % stride else 0))
    
    total_tiles = h_tiles * w_tiles
    print(f"Processing {total_tiles} tiles ({h_tiles}x{w_tiles})...")
    
    tile_count = 0
    for i in range(h_tiles):
        for j in range(w_tiles):
            # Calculate tile boundaries
            y1 = min(i * stride, h - tile_size) if h > tile_size else 0
            x1 = min(j * stride, w - tile_size) if w > tile_size else 0
            y2 = min(y1 + tile_size, h)
            x2 = min(x1 + tile_size, w)
            
            # Extract and process tile
            tile = img_tensor[:, :, y1:y2, x1:x2]
            
            with torch.no_grad():
                tile_out = model(tile)
            
            # Calculate output positions
            out_y1, out_y2 = y1 * scale, y2 * scale
            out_x1, out_x2 = x1 * scale, x2 * scale
            
            # Create weight mask for blending
            tile_h, tile_w = tile_out.shape[2:]
            mask = torch.ones((1, 1, tile_h, tile_w), device=tile_out.device, dtype=tile_out.dtype)
            
            # Feather edges for blending
            feather = tile_overlap * scale // 2
            if feather > 0:
                # Top edge
                if i > 0:
                    for k in range(feather):
                        mask[:, :, k, :] *= k / feather
                # Bottom edge
                if i < h_tiles - 1:
                    for k in range(feather):
                        mask[:, :, -(k+1), :] *= k / feather
                # Left edge
                if j > 0:
                    for k in range(feather):
                        mask[:, :, :, k] *= k / feather
                # Right edge
                if j < w_tiles - 1:
                    for k in range(feather):
                        mask[:, :, :, -(k+1)] *= k / feather
            
            # Add to output with blending
            output[:, :, out_y1:out_y2, out_x1:out_x2] += tile_out * mask
            weight[:, :, out_y1:out_y2, out_x1:out_x2] += mask
            
            tile_count += 1
            print(f"  Processed {tile_count}/{total_tiles} tiles", end='\r')
    
    print(f"  Processed {tile_count}/{total_tiles} tiles")
    
    # Normalize by weights
    output = output / weight.clamp(min=1e-8)
    
    return output


def save_image_formats(img_array: np.ndarray, output_name: str, output_dir: Path, formats: list) -> list:
    """Save image in multiple formats.
    
    Args:
        img_array: Image as numpy array (H, W, C) in RGB format, uint8
        output_name: Base filename without extension
        output_dir: Output directory
        formats: List of formats to save, e.g. ["png", "tiff"]
    
    Returns:
        List of saved file paths
    """
    output_dir.mkdir(parents=True, exist_ok=True)
    pil_img = Image.fromarray(img_array)
    
    saved_paths = []
    for fmt in formats:
        fmt_lower = fmt.lower()
        if fmt_lower == "png":
            output_path = output_dir / f"{output_name}.png"
            pil_img.save(str(output_path), "PNG")
            saved_paths.append(str(output_path))
            print(f"   Saved: {output_path.name}")
        elif fmt_lower in ["tiff", "tif"]:
            output_path = output_dir / f"{output_name}.tiff"
            pil_img.save(str(output_path), "TIFF", compression=None)
            saved_paths.append(str(output_path))
            print(f"   Saved: {output_path.name}")
    
    return saved_paths


def upscale_and_resize(
    image_path: str,
    target_width_inches: float,
    target_height_inches: float,
    dpi: int,
    output_name: str,
    output_formats: list,
    model_name: str,
    tile_size: int,
    tile_overlap: int,
    use_fp16: bool,
    use_two_pass: bool
) -> dict:
    """
    Upscale image with Real-ESRGAN using aspect-ratio-aware scaling.
    
    The output will be sized so that after human cropping to the target
    aspect ratio, the image will have exactly the target DPI.
    
    Args:
        image_path: Path to input image
        target_width_inches: Target print width in inches
        target_height_inches: Target print height in inches
        dpi: Target DPI for printing
        output_name: Output filename (without extension)
        output_formats: List of formats to save, e.g. ["png", "tiff"]
        model_name: Name of upscale model
        tile_size: Tile size for processing
        tile_overlap: Overlap between tiles
        use_fp16: Use FP16 precision
        use_two_pass: If True, run 4x model twice for 16x total
    
    Returns:
        dict with output paths and crop info
    """
    start_time = time.time()
    
    # Resolve image path
    image_path = Path(image_path)
    if not image_path.is_absolute():
        image_path = REPO_ROOT / image_path
    
    if not image_path.exists():
        raise FileNotFoundError(f"Input file not found: {image_path}")
    
    print(f"Processing: {image_path.name}")
    print(f"Target print size: {target_width_inches}\" x {target_height_inches}\" @ {dpi} DPI")
    
    # Load image
    print("Loading image...")
    img = cv2.imread(str(image_path))
    if img is None:
        raise ValueError(f"Could not load image: {image_path}")
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]
    input_aspect = w / h
    target_aspect = target_width_inches / target_height_inches
    print(f"Input size: {w}x{h} (aspect: {input_aspect:.3f})")
    print(f"Target aspect: {target_aspect:.3f}")
    
    # Calculate aspect-ratio-aware output dimensions
    scale_info = calculate_scale_for_crop(
        input_width=w,
        input_height=h,
        target_width_inches=target_width_inches,
        target_height_inches=target_height_inches,
        dpi=dpi
    )
    
    output_width = scale_info["output_width_px"]
    output_height = scale_info["output_height_px"]
    final_width = scale_info["final_width_px"]
    final_height = scale_info["final_height_px"]
    
    print(f"\n📐 Scaling Strategy:")
    print(f"   Pre-crop output: {output_width}x{output_height}px ({scale_info['equivalent_print_size']})")
    print(f"   Final after crop: {final_width}x{final_height}px ({target_width_inches}\" x {target_height_inches}\")")
    if scale_info["crop_direction"] != "none":
        print(f"   Crop needed: {scale_info['crop_amount_px']}px {scale_info['crop_direction']} ({scale_info['crop_amount_inches']:.2f}\")")
    
    # Calculate effective scale after upscaling
    effective_scale = 16 if use_two_pass else 4
    upscaled_w, upscaled_h = w * effective_scale, h * effective_scale
    
    if use_two_pass:
        print(f"\n   Step 1: Two-pass 4x upscale (16x total) -> {upscaled_w}x{upscaled_h}")
    else:
        print(f"\n   Step 1: 4x upscale -> {upscaled_w}x{upscaled_h}")
    print(f"   Step 2: Resize to pre-crop size -> {output_width}x{output_height}")
    
    # Convert to tensor
    img_tensor = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).float() / 255.0
    img_tensor = img_tensor.to(DEVICE)
    
    if use_fp16 and DEVICE.type == "cuda":
        img_tensor = img_tensor.half()
    
    # Load model
    print(f"\nLoading model: {model_name}")
    model_path = UPSCALE_MODELS_DIR / model_name
    if not model_path.exists():
        raise FileNotFoundError(f"Model not found: {model_path}")
    
    model = ModelLoader().load_from_file(str(model_path))
    assert isinstance(model, ImageModelDescriptor), "Not an image model!"
    
    model = model.to(DEVICE)
    if use_fp16 and DEVICE.type == "cuda":
        model = model.half()
    model.eval()
    
    scale = model.scale
    print(f"Model scale: {scale}x")
    
    # First pass upscale
    print("Upscaling (pass 1 of {})...".format(2 if use_two_pass else 1))
    upscale_start = time.time()
    
    output_tensor = upscale_with_tiles(model, img_tensor, tile_size, tile_overlap)
    
    pass1_time = time.time() - upscale_start
    print(f"Pass 1 took: {pass1_time:.1f}s")
    
    # Second pass if requested
    if use_two_pass:
        print("Upscaling (pass 2 of 2)...")
        pass2_start = time.time()
        
        # Use smaller tile size for second pass (image is now 4x larger)
        tile_size_pass2 = min(tile_size, 384)  # Smaller tiles for larger image
        output_tensor = upscale_with_tiles(model, output_tensor, tile_size_pass2, tile_overlap)
        
        pass2_time = time.time() - pass2_start
        print(f"Pass 2 took: {pass2_time:.1f}s")
    
    total_upscale_time = time.time() - upscale_start
    print(f"Total upscaling took: {total_upscale_time:.1f}s")
    
    # Convert back to image
    output = output_tensor.squeeze(0).permute(1, 2, 0).float().cpu().numpy()
    output = (output * 255).clip(0, 255).astype(np.uint8)
    
    actual_upscaled_h, actual_upscaled_w = output.shape[:2]
    print(f"Upscaled size: {actual_upscaled_w}x{actual_upscaled_h}")
    
    # Resize to pre-crop dimensions using high-quality Lanczos
    print(f"Resizing to pre-crop size: {output_width}x{output_height}...")
    pil_img = Image.fromarray(output)
    pil_img = pil_img.resize((output_width, output_height), Image.LANCZOS)
    
    # Convert back to numpy (RGB)
    output_rgb = np.array(pil_img)
    
    # Save output in all requested formats
    print(f"\nSaving outputs ({', '.join(output_formats)})...")
    output_paths = save_image_formats(output_rgb, output_name, OUTPUT_DIR, output_formats)
    
    # Cleanup
    del model, img_tensor, output_tensor
    clear_memory()
    
    total_time = time.time() - start_time
    
    print(f"\n✅ Done!")
    print(f"Output size: {output_width}x{output_height}px")
    print(f"Equivalent print size: {scale_info['equivalent_print_size']} @ {dpi} DPI")
    
    if scale_info["crop_direction"] != "none":
        print(f"\n📋 CROP INFO FOR HUMAN:")
        print(f"   Direction: {scale_info['crop_direction'].upper()}")
        print(f"   Amount to crop: {scale_info['crop_amount_px']}px ({scale_info['crop_amount_inches']:.2f}\")")
        print(f"   After crop: {final_width}x{final_height}px = {target_width_inches}\" x {target_height_inches}\" @ {dpi} DPI")
    
    print(f"\nTotal time: {total_time:.1f}s")
    
    return {
        "output_paths": output_paths,
        "output_width": output_width,
        "output_height": output_height,
        "crop_direction": scale_info["crop_direction"],
        "crop_amount_px": scale_info["crop_amount_px"],
        "crop_amount_inches": scale_info["crop_amount_inches"],
        "final_width": final_width,
        "final_height": final_height,
        "equivalent_print_size": scale_info["equivalent_print_size"]
    }

print("✅ Functions defined!")

✅ Functions defined!


In [6]:
# @title Run 2x Evaluation Upscaling

print("="*60)
print("2x EVALUATION MODE - REAL-ESRGAN UPSCALING")
print("="*60)

results = []

for i, config in enumerate(IMAGE_CONFIGS, 1):
    print(f"\n{'='*60}")
    print(f"[{i}/{len(IMAGE_CONFIGS)}] {config['image_id']}")
    print(f"Original: {config['original_width']}\" x {config['original_height']}\"")
    print(f"Evaluating at: {config['eval_width_inches']}\" x {config['eval_height_inches']}\"")
    print(f"{'='*60}")
    
    try:
        # Use _2x-eval suffix for evaluation outputs
        output_name = f"{config['image_id']}_real-esrgan_2x-eval"
        
        result = upscale_and_resize(
            image_path=config["input_path"],
            target_width_inches=config["eval_width_inches"],
            target_height_inches=config["eval_height_inches"],
            dpi=DPI,
            output_name=output_name,
            output_formats=OUTPUT_FORMATS,
            model_name=upscale_model,
            tile_size=tile_size,
            tile_overlap=tile_overlap,
            use_fp16=use_fp16,
            use_two_pass=USE_TWO_PASS
        )
        
        results.append({
            "image_id": config["image_id"],
            "status": "Success",
            "outputs": result["output_paths"],
            "output_size": f"{result['output_width']}x{result['output_height']}",
            "eval_size": result["equivalent_print_size"],
            "original_target": f"{config['original_width']}\" x {config['original_height']}\""
        })
        
    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()
        results.append({
            "image_id": config["image_id"],
            "status": "Failed",
            "error": str(e)
        })

# Summary
print(f"\n\n{'='*60}")
print("2x EVALUATION - COMPLETE")
print(f"{'='*60}")

success_count = sum(1 for r in results if r["status"] == "Success")
print(f"\nProcessed: {success_count}/{len(results)} images successfully\n")

for r in results:
    if r["status"] == "Success":
        print(f"✅ {r['image_id'][:8]}...")
        for out_path in r["outputs"]:
            print(f"   Output: {Path(out_path).name}")
        print(f"   Size: {r['output_size']} ({r['eval_size']} @ {DPI} DPI)")
        print(f"   Original target: {r['original_target']}")
        print()
    else:
        print(f"❌ {r['image_id'][:8]}... -> {r.get('error', 'Unknown error')}")

print("\n📋 Next steps:")
print("   1. Open the output in Photoshop")
print("   2. View at 100% to check for pixelation")
print("   3. If quality is acceptable, run the standard notebook for final output")

2x EVALUATION MODE - REAL-ESRGAN UPSCALING

[1/3] 7a49a140-beee-4141-9593-3fddb9194240
Original: 20" x 10"
Evaluating at: 40" x 20"
Processing: 7a49a140-beee-4141-9593-3fddb9194240.jpg
Target print size: 40" x 20" @ 150 DPI
Loading image...
Input size: 682x1024 (aspect: 0.666)
Target aspect: 2.000

📐 Scaling Strategy:
   Pre-crop output: 6000x9008px (40" x 60.05")
   Final after crop: 6000x3000px (40" x 20")
   Crop needed: 6008px vertical (40.05")

   Step 1: Two-pass 4x upscale (16x total) -> 10912x16384
   Step 2: Resize to pre-crop size -> 6000x9008

Loading model: 4x-UltraSharp.pth
Model scale: 4x
Upscaling (pass 1 of 2)...
Processing 6 tiles (3x2)...
  Processed 6/6 tiles
Pass 1 took: 8.3s
Upscaling (pass 2 of 2)...
Processing 96 tiles (12x8)...
  Processed 96/96 tiles
Pass 2 took: 68.9s
Total upscaling took: 77.2s
Upscaled size: 10912x16384
Resizing to pre-crop size: 6000x9008...

Saving outputs (png, tiff)...
   Saved: 7a49a140-beee-4141-9593-3fddb9194240_real-esrgan_2x-eval.pn

## Single Image Processing (Optional)

Use the cell below if you want to process a single image with custom settings.

In [7]:
# @title Single Image Upscale (Optional)

# Uncomment and modify to process a single image:

# single_result = upscale_and_resize(
#     image_path="input/your_image.jpg",
#     target_width=1650,   # 11 inches * 150 DPI
#     target_height=2100,  # 14 inches * 150 DPI
#     output_name="your_image_real-esrgan",
#     model_name="4x-UltraSharp.pth",
#     tile_size=512,
#     tile_overlap=32,
#     use_fp16=True
# )